<a href="https://colab.research.google.com/github/penelopetir/BIOL74-Final-Project/blob/main/Final_Project_Penelope_and_Andy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before starting, we'll install neuron in our current runtime as usual.

In [1]:
%pip install neuron # only need to run this cell once to install
                    # neuron in the local jupyter environment

Run the code block below just once to get all the files from the repository into our colab session and compile the MOD mechanism files we'll be using

In [8]:
repo_name = 'BIOL74-Final-Project'
if 'google.colab' in str(get_ipython()):
    import os
    if not os.path.exists(repo_name):
        !git clone https://github.com/penelopetir/{repo_name}.git # downloads repository into our Google colab session's file system

    os.chdir(repo_name) # Changing working directory to downloaded repository

# Compile mechanisms
!nrnivmodl mechanisms

Cloning into 'BIOL74-Final-Project'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 76 (delta 31), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (76/76), 56.36 KiB | 1.22 MiB/s, done.
Resolving deltas: 100% (31/31), done.
/content/BIOL74-Final-Project/BIOL74-Final-Project
Mod files: "mechanisms/mechanisms/CaDynamics_E2.mod"

Creating 'x86_64' directory for .o files.

 -> Compiling mod_func.cpp
 -> NMODL ../mechanisms/CaDynamics_E2.mod
Translating CaDynamics_E2.mod into /content/BIOL74-Final-Project/BIOL74-Final-Project/x86_64/CaDynamics_E2.c
Thread Safe
 -> Compiling CaDynamics_E2.c
 => LINKING shared library ./libnrnmech.so
 => LINKING executable ./special LDFLAGS are:    -pthread
Successfully created x86_64/special


## Introduction
For this exercise, we'll be adapting a model of a L5PC to model granule neurons traced from wild type and PTEN KO groups


Start by importing the packages we'll be using.

In [4]:
from neuron import h # all NEURON hoc functions are accessed through h
from neuron import gui # if you're running this notebook in a local environment (instead of on google colab), launches the GUI
h.load_file('stdrun.hoc') # loads neuron standard hoc library, not always necessary
import numpy as np
import matplotlib as mpl
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
import matplotlib.pyplot as plt
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

if 'google.colab' in str(get_ipython()): # For making interactive plots work on google colab
    from google.colab import output
    output.enable_custom_widget_manager()
else:
    plotly.offline.init_notebook_mode() # for printing notebooks offline

# Dendritic spines and NMDA spikes

Let's first explore how dendritic inputs to different parts of a dendritic arbor impact both the local membrane potential in the dendrite and the somatic membrane potential, which is ultimately what needs to reach threshold to output an axonal action potential.

We start by importing the necessary model files to instantiate the neuron model with a helper function called `createL5PC`. This python function takes 4 arguments:

1) `morphology_file`: a string containing the name of the morphology file

2) `spine_seclist_name`: the [SectionList](https://www.neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/topology/seclist.html) of the dendritic branch we want to populate with spines. `seclist_name` name should be a string that's either `'basal'` for a basal dendritic branch or `'apical'` for an apical dendritic branch. In order to accomodate the additional spine compartments, this function increases the number of compartments in this segment to have at least 2 segments per spine (which comes with added computational cost). Note: the section list names are different from the section names; `apical` sections are named `apic`, e.g., `apic[0]`,`apic[1]`, etc., and `basal` sections are named `dend`, e.g., `dend[0]`,`dend[1]`, etc.

3) `spine_sec_ind`: the index of the section within that list to populate with spines

4) `spine_interv`: the interval at which the spines will occur in $\mu m$.

It then returns the cell object. It also deletes any existing cells before creating a new one, which is worth keeping in mind in case you want to use it to generate multiple cells in the future. Arguments 2–4 are input to `add_spines` which then creates spines consisting of two single segment `Sections`, `spine_neck` and `spine_head` that are attached to the parent dendritic branch you chose. Each `spine_neck` section is added to the `spine_necks` `SectionList`, and each `spine_head` section is added to the `spine_heads` `SectionList`.

In the code block below, select one of the following branches to populate with dendritic spines, you'll be able to visualize which branch this is below:

1) `L5PC.apic[36]` - part of the main apical dendritic branch

2) `L5PC.apic[67]` - a branch in the apical tuft

3) `L5PC.dend[13]` - a basal dendritic branch

4) `L5PC.apic[9]` - an oblique apical dendritic branch

Populate this branch with a spine every $2\; \mu m$.

In [9]:
from helper_functions import createL5PC

# fill in code here
spine_seclist_name = 'apical'
spine_sec_ind = 1 # specify index of branch within section list
spine_interv = 2 # spacing between spines in µm

WT_file = "morphologies/WT.asc" # morphology file
KO_file = "morphologies/PTEN_KO.asc" # morphology file
WT = createL5PC(WT_file,spine_seclist_name,spine_sec_ind,spine_interv)
PTEN_KO = createL5PC(KO_file,spine_seclist_name,spine_sec_ind,spine_interv)

# Create recording vectors for time and somatic voltage
t_vec = h.Vector().record(h._ref_t)
v_soma_WT = h.Vector().record(WT.soma[0](0.5)._ref_v)
v_soma_KO = h.Vector().record(PTEN_KO.soma[0](0.5)._ref_v)

NEURON: soma : section was deleted
 in L5PCbiophys.hoc near line 0
 }
  ^
        L5PCtemplate[0].geom_nseg()
      L5PCtemplate[0].init("morphologi...", "apical", 1, 2)


RuntimeError: hocobj_call error

### Add one AMPA/NMDA synapse to each spine

The code below generates one AMPA/NMDA synapse (`ProbAMPANMDA`) per spine with an associated `NetCon` and `NetStim` object, which are each appended to lists. For a refresher on these classes, see [Exercise 9](https://github.com/CompModNervSystem/CompModNervSys-BallAndStickSynapses/blob/main/CompModNervSys_exercise9.ipynb), and/or the documentation for [`NetCon`](https://www.neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/network/netcon.html) and [`NetStim`](https://www.neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/mechanisms/mech.html#NetStim).

Last exercise we turned the NMDA current off to isolate the contribution of the fast AMPA receptor conductance.  Now, let's explore how the combination of AMPA and NMDA receptors shapes the postsynaptic response. We'll use an NMDA/AMPA ratio of 0.71, but this ratio varies between specific typs of synaptic connections.

In [ ]:
# Create a synapse for each spine
synapses = [] # Store synapses
netstims = [] # Store NetStims
netcons = [] # Store NetCons
# recording vectors for voltage and current at each synapse (spine head)
v_syns = [] # voltage recordings
i_syns = [] # synaptic current recordings
i_AMPAs = [] # AMPAr specific currents
i_NMDAs = [] # AMPAr specific currents
synapse_names = [] # string names for each synapse
for i in range(int(L5PC.nSecSpines)):
    synapses.append(h.ProbAMPANMDA(L5PC.spine_head[i](0.5)))
    netstims.append(h.NetStim())
    netcons.append(h.NetCon(netstims[i],synapses[i]))
    v_syns.append(h.Vector().record(L5PC.spine_head[i](0.5)._ref_v))
    i_syns.append(h.Vector().record(synapses[i]._ref_i))
    i_AMPAs.append(h.Vector().record(synapses[i]._ref_i_AMPA))
    i_NMDAs.append(h.Vector().record(synapses[i]._ref_i_NMDA))
    synapse_names.append('spine[{}]'.format(i))

### Define synaptic parameters

The `setParams` function defined below allows you to set the values of any of the synapse objects (defined as `ProbAMPANMDA` mechanisms), `NetStim` objects, or `NetCon` objects. The syntax for these objects is the same as we used in the last exercise.

The code cell below uses `setParams` to set all the `NetCon` weights to 0.5, which gives $\bar{g}_{AMPA} \approx 0.5 \; nS$. The weight of each NMDA conductance is set to 0.71 times the AMPA conductance by default. It then sets the number of `NetStim` events (artificial spikes) to 0, turning them off for now.

In [ ]:
def setParams(obj_list, indices,settings):
    # obj_list - list of synapses, netstims, or netcons
    # indices - array of indices (integers) of NetStims within the lists to modify
    # settings - dictionary of key, value pairs, should match fields of object in list
    for i in indices:
        obji = obj_list[i]
        for key, val in settings.items():
            h('{}.{} = {}'.format(obji.hname(),key,val))

# Set all the weights to 0.5 (peak conductance of 0.5 nS) and delays to 1 ms
setParams(netcons,range(len(synapses)),{'weight':0.5,'delay':1})
# turn off netstims for now
setParams(netstims,range(len(synapses)),{'number':0})

### Visualize the morphology

Now let's visualize our cell's morphology and the location of the synapses we added.

In [ ]:
ps = h.PlotShape(False) # generate a NEURON PlotShape object
fig = ps.plot(plotly) # Plot using plotly's renderer
scene=dict(camera=dict(up=dict(x=0,y=1,z=0),eye=dict(x=2,y=2,z=2.5)), # define view properties
           xaxis=dict(nticks=3,showticklabels=True),
           yaxis=dict(nticks=3,showticklabels=True),
           zaxis=dict(nticks=3,showticklabels=True),
           aspectmode='data', #this string can be 'data', 'cube', 'auto', 'manual'
           )
fig.update_layout(scene=scene,autosize=False,width=400,height=600)
for syn in synapses: # Mark each synapse
    fig.mark(syn.get_segment())
fig.update_layout(title='Synapse locations')
plotly.io.show(fig)


### Amplification of EPSPs in single spines

Let's first try turning on a single synapse. For these simulations, we'll want to run the simulation for some time to let the voltage equilibriate throughout the model, since we now have several non-linear voltage gated ion channels interacting near rest with non-uniform distributions in the cell.

We'll create a plotting function below to visualize the output.

In [ ]:
def plot_V_recs(t_vec,v_soma,v_recs,rec_names,x_lim=None,title=None):
    # fig = plt.figure()
    # ax = fig.add_subplot(111)
    # ax.plot(t_vec,v_soma,label = 'soma')
    # for v,name in zip(v_recs,rec_names):
    #     ax.plot(t_vec,v,label=name) # plots last simulation
    # ax.set_xlabel('time (ms)')
    # ax.set_ylabel('Vm (mV)')
    # ax.legend(frameon=False)
    # if x_lim is not None:
    #     ax.set_xlim(x_lim)
    fig = make_subplots(rows=1,cols=1)
    fig.add_trace(go.Scatter(x=t_vec,y=v_soma,name='soma',line=dict(color='rgb(0,0,0)')))
    for v, name in zip(v_recs,rec_names):
        fig.add_trace(go.Scatter(x=t_vec,y=v,name=name))
    fig['layout']['yaxis']['title'] = 'Vm (mV)'
    fig['layout']['xaxis']['title'] = 'time (ms)'
    if x_lim is not None:
        fig.update_layout(xaxis_range=x_lim)
    if title is not None:
        fig.update_layout(title=title)
    fig.show()
    return fig

Now, using `setParams`, set `number` to 1 and `start_time` to $150 \; ms$ for the first synapse in your `synapses` list.
Run the simulation for $200 \; ms$ by setting `h.tstop`, set the initial voltage (`h.v_init`) to $-80 mV$, and set the temperature to $37 ^{\circ} C$

In [ ]:
# your code goes here
synapse_index =
start_time =

setParams(netstims,[synapse_index],{'number':,'start':}) # activate 1 AP per synapse

# Simulation settings
h.celsius =  # deg C
h.tstop =  # ms
h.v_init =  # mV

# Run the simulation
h.run()

# Plot
fig = plot_V_recs(t_vec,v_soma,[v_syns[synapse_index]],[synapse_names[synapse_index]],x_lim=(start_time-10,start_time+50));


You should see a sizable EPSP on the spine that is attenuated significantly at the soma. How does the presence of the spine alter the local dendritic voltage, i.e., what's the voltage in the parent dendritic branch?

We can automatically identify and record from this branch by using some handy methods of NEURON's `Section` and `Segment` python classes, shown below.

In [ ]:
parent_seg = synapses[synapse_index].get_segment().sec.parentseg().sec.parentseg() # spine head-> spine neck -> dendritic branch
v_parent_dend = h.Vector().record(parent_seg._ref_v) # record from parent dendritic branch

h.run() # Run again now that we have a new recording in place

plot_V_recs(t_vec,v_soma,[v_syns[synapse_index],v_parent_dend],[synapse_names[synapse_index],'parent_branch'],x_lim=(start_time-10,start_time+50));


<font color='red'>**Q1**: What electrical property of the dendritic spine causes this local amplification?
</font>

<font color='red'>**Q2**: What happens if you set the spine neck axial resistance to a low value, e.g., $1 \; \Omega cm$, and why? Try it out in the cell below and answer in the subsequent text cell. Hint: think back to the voltage divider circuit from [Exercise 2](https://github.com/CompModNervSystem/CompModNervSys-BioElectroStatics)!
</font>

In [ ]:
# Change Ra here
# your code goes here

h.run() # Run again now that we have a new recording in place

plot_V_recs(t_vec,v_soma,[v_syns[synapse_index],v_parent_dend],[synapse_names[synapse_index],'parent_branch'],
            x_lim=(start_time-10,start_time+50));

L5PC.spine_neck[synapse_index].Ra = 100 # resets Ra to default value

### Threshold for NMDA spike generation

Now let's try simulating increasing levels of excitation. This can come in the form of additional synapses being synchronously activated or each synapse being activated with additional action potentials in a short time span. The possible parameter space is vast (number and spatial distribution of synapses, frequency and pattern of inputs, etc.), so we'll just explore a few ways this could occur.

To make things easier, I've defined a function below called `turnOnSynapses` that allows turning on a given number of synapses (`n_syn`) within a list of synapses, according to one of two options for `distribute_mode`: `'sequential'` or `'even'`. The `NetStim` objects associated with each synapse are all assigned parameters defined in a dictionary `netstim_params`. See the comments in the code below for an example.


In [ ]:
def turnOnSynapses(n_syn,distribute_mode,netstims,netstim_params):
    # Turn on a set number of synapses using setParams
    # Input arguments:
    #   n_syn - number of synapses to turn on
    #   distribute_mode - 'sequential' or 'even', 'sequential' turns on synapses sequentially from proximal to distal in section (e.g.,
    #                       with n_syn = 3, the 1st, 2nd, and 3rd synapse would be turned on)
    #                      'even' turns on synapses distributed evenly throughout the section (e.g. with n_syn = 3, the synapse at the
    #                       beginning, middle, and end of the section would be turned on)
    #   netstims - list of NetStims connected to each synapse
    #   netstim_params - dictionary of NetStim parameters to assign, should at least include 'number' to set the number of APs to activate
    # Example:
    # turnOnSynapses(10,'even',netstims,{'number':1,'start':100}) # turns on 10 synapses, evenly distributed, with 1 AP delivered at 100 ms

    setParams(netstims,range(len(netstims)),{'number':0}) # turn all off to initialize
     # generate indices of synapses to turn on
    if distribute_mode == 'sequential': # turns on synapses sequentially from proximal to distal in section
        if n_syn > 1:
            syn_indices = range(n_syn)
        else:
            syn_indices = [0]
    elif distribute_mode == 'even': # turns on synapses evenly distributed in section
        if n_syn > 1:
            syn_indices = list(np.linspace(0,len(netstims)-1,n_syn,dtype=int))
        else:
            syn_indices = [int(len(netstims)/2)]
    setParams(netstims,syn_indices,netstim_params) # assign parameters for all synpases in syn_indices

    return syn_indices

You're now ready to run some computational experiments. Let's start by addressing the question: how many synapses are required, i.e. what is the threshold, to generate an NMDA spike in this specific dendritic branch?

Use the code block below to change the number of synapses, their distribution, and the presence of NMDA. As you increase the number of synapses, you should see an NMDA spike. How does activating synapses clustered together sequentially vs. evenly distributed along the branch alter the net effect? What happens to the somatic potential?

In [ ]:
n_synapses_on = 1
distribute_mode = 'even'
start_time = 150
netstims_params = {'number':1,'start':start_time}
h.tstop = 300
# Turn on NMDA
setParams(synapses,range(len(synapses)),{'NMDA_ratio':0.71})
# Turn on desired number of synapses
syn_indices = turnOnSynapses(n_synapses_on,distribute_mode,netstims,netstims_params)
# Run simulation
h.run()
# Plot
v_soma0 = np.array(v_soma) # grab soma voltage for use later
v_syns0 = [np.array(v) for v in v_syns] # grab synaptic voltages for use later
plot_V_recs(t_vec,v_soma,[v_syns[i] for i in syn_indices],[synapse_names[i] for i in syn_indices],x_lim=(start_time-10,h.tstop));

How might you test that what you observe is a true NMDA spike? In the code block below, run a computational experiment to address this question. Hint: You may want to use `setParams`...

In [ ]:
# Use setParams here

# Run simulation
h.run()

# Plot

plot_V_recs(t_vec,v_soma,[v_syns[i] for i in syn_indices],[synapse_names[i] for i in syn_indices],x_lim=(start_time-10,h.tstop));

Is it possible other voltage-gated conductances are involved in the dendritic branch you're stimulating?

The `toggleChannelSeclist` function below allows you to turn off (or back on) any of the channels present in the model in one of the subregions, which are organized into `SectionList`s. The possible section lists are available within the cell object (`L5PC` in this case) and include `L5PC.apical`, `L5PC.basal`, `L5PC.somatic`, or `L5PC.axonal`. The spines also have channels placed in the `spine_head` section mimicking their parent branch, which are included in the `spine_heads` `SectionList`.

To specify the channels to switch off/on, I've simplified things a bit to allow you to set groups of channels by setting `channel_type` to one of the following strings:

- `'Cav'` - voltage-gated calcium channels, sections may include a low-voltage activated and high-voltage activated channel
- `'Kv'` - voltage-gated potassium channels, sections may include a Kv3 channel, and a transient and persistent potassium conductance (only present in the soma)
- `'Nav'` - voltage-gated sodium cahnnels, sections may include a transient (inactivating) and persistent (non-inactivating) Nav conductance
- `'Kca'` - calcium-activated potassium channel
- `'Ih'` - hyperpolarization activated current

In [ ]:
def toggleChannelSeclist(channel_type,cell,seclist,turn_off):
    # Turns off/on set of ion channel conductances in input seclist
    # Input arguments:
    #   channel_type - string of channel type to turn off, see key of mech_names dictionary for possible channel types
    #   cell - Cell template object
    #   seclist - NEURON h.SectionList object
    #   turn_off - True or False. True to turn off channels (set gbar to 0), False to revert channels back to default conductances
    # example:
    # L5PC = h.L5PCtemplate(morphology_file)
    # toggleChannelSeclist('Cav',L5PC,L5PC.apical,1) # turns off Cav channels
    # toggleChannelSeclist('Cav',L5PC,L5PC.apical,0) # turns back on
    mech_names = {
        'Cav': [('gCa_LVAstbar','Ca_LVAst'),('gCa_HVAbar','Ca_HVA')],
        'Kv': [('gK_Tstbar','K_Tst'),('gK_Pstbar','K_Pst'),('gSKv3_1bar','SKv3_1'),('gImbar','Im')],
        'Nav': [('gNaTa_tbar','NaTa_t'),('gNap_Et2bar','Nap_Et2')],
        'Kca': [('gSK_E2bar','SK_E2')],
        'Ih': [('gIhbar','Ih')]
    }
    if turn_off:
        for sec in seclist: # loop through sections in section list
            for gbar,mech in mech_names[channel_type]: # grab the name of the peak conductance and mechanism
                if h.ismembrane(mech,sec=sec): # check if mechanism is present in this section
                    h('{} {}_{} = 0'.format(sec.hname(),gbar,mech)) # turn off conductance
        print('Turned off {} currents in {} sections'.format(channel_type,seclist))
    else:
       cell.biophys() # resets conductances to default values
       print('Reverted conductances back to default values')


You can test the contribution of any of these channel types in the code block below

In [ ]:
n_synapses_on = 50
distribute_mode = 'even'
start_time = 150
netstims_params = {'number':1,'start':start_time}
EPSP_peaks = []
h.tstop = 300
# Turn on NMDA
setParams(synapses,range(len(synapses)),{'NMDA_ratio':0.71})

# Turn on desired number of synapses
syn_indices = turnOnSynapses(n_synapses_on,distribute_mode,netstims,netstims_params)

# Call toggleChannelSeclist here, set to Nav currents to start, modify to channel of your choice
toggleChannelSeclist('Nav',L5PC,L5PC.apical,turn_off=True)

# Run simulation
h.run()
# Plot
v_soma0 = np.array(v_soma) # grab soma voltage for use later
v_syns0 = [np.array(v) for v in v_syns] # grab synaptic voltages for use later
plot_V_recs(t_vec,v_soma,[v_syns[i] for i in syn_indices],[synapse_names[i] for i in syn_indices],x_lim=(start_time-10,h.tstop));

<font color='red'>**Q3**: Summarize your findings from these computational experiments. In your answer, you should address the questions posed above: how many synapses were necessary to generate an NMDA spikes? How did the net output change if you activated synapses clustered together (sequentially) vs. evenly distributed along the branch? How was the somatic potential affected? Also describe the results of your experiments to verify the NMDA spike and identify the contribution of ion channels to the response.
</font>

### Reflections

You now have all the software tools to probe the model and run virtually any experiment you can imagine. How does the somatic EPSP vary with number of synapses, and how does this vary with their spatial arrangement? Do synapses clustered together vs. broadly distributed sum differently? How does this summation vary if synapses are made onto spines vs. shafts or with the ratio of AMPA to NMDA currents? And ultimately, how might these biophysical properties constrain, or enable, the processing of information received via presynaptic inputs? Of course, with a model this complex, we should always take caution and make sure what we observe agrees with with real experimental data (when available) and our physical intuition.

[Part 2](https://github.com/CompModNervSystem/CompModNervSys-NonLinearDendrites/blob/main/CompModNervSys_exercise10_part2.ipynb) explores a mechanism hypothesized to enable pyramidal neurons to perform *coincidence detection*, allowing them to associate different inputs arriving within a narrow time window by generating bursts of action potentials.